In [1]:
import pandas as pd
INPUT_PATH = '../../datasets/GoogleAmazonSampled_400.csv'
OUTPUT_PATH = '../../evals/eval_Google_Amazon_v1.csv'

df_data = pd.read_csv(INPUT_PATH, index_col=False)
df_eval = pd.read_csv(OUTPUT_PATH, index_col=False)

PROMPT_NAME = "match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301"
EVAL_COLUMN_NAME = "match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301_maxtok-15"
MODEL = "openAI - gpt-3.5-turbo-0301"

DATASET_NAME = "GoogleAmazon"



In [2]:
df_data.head(10)

,Unnamed: 0,ltable_id,rtable_id,label,prompt1_en_short_temp_0_gpt3.5-0301,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301
0,0,363,2854,True,True,True
1,1,1269,1433,True,True,True
2,2,63,2129,True,True,False
3,3,540,5,True,True,True
4,4,1262,192,True,True,True
5,5,1320,685,True,False,False
6,6,1092,978,True,True,True
7,7,1214,549,True,False,True
8,8,1026,2327,True,False,False
9,9,594,606,True,True,True


In [3]:
from sklearn import metrics
import datetime


def evalColumn(model, prompt, df, column):
    y_true = df["label"]
    y_pred = df[column]
    conf_matrix = metrics.confusion_matrix(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    accuracy = metrics.accuracy_score(y_true, y_pred)
    global df_eval
    df_eval = df_eval._append({'Model': model, 'Prompt': prompt, 'Accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, "confusion_matrix": conf_matrix, "Dataset":DATASET_NAME, "Time":datetime.datetime.now()}, ignore_index=True)

evalColumn(MODEL, EVAL_COLUMN_NAME, df_data, PROMPT_NAME)
    

In [4]:
print(df_data["label"])

0       True
1       True
2       True
3       True
4       True
       ...  
395    False
396    False
397    False
398    False
399    False
Name: label, Length: 400, dtype: bool


In [5]:
print(df_data[PROMPT_NAME])

0       True
1       True
2      False
3       True
4       True
       ...  
395    False
396    False
397    False
398    False
399     True
Name: match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301, Length: 400, dtype: bool


In [6]:
df_eval

,Model,Prompt,Accuracy,precision,recall,F1,confusion_matrix,Dataset,Time
0,openAI - gpt-3.5-turbo-0301,prompt1_en_short_temp_0_gpt3.5-0301_maxtok-15,0.9125,0.545455,0.75,0.631579,[[335 25]\n [ 10 30]],GoogleAmazon,2023-10-22 16:59:39.414990
1,openAI - gpt-3.5-turbo-0301,match_prompt_manual_fewshot_1_eval2_temp_0_gpt...,0.8525,0.379747,0.75,0.504202,"[[311, 49], [10, 30]]",GoogleAmazon,2023-10-23 11:53:13.652323


In [7]:
# write to csv
df_eval.to_csv(OUTPUT_PATH, index=False)